In [3]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup as bs

In [4]:
print_url = 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId='

In [5]:
url = "https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=84546"

In [6]:
response = requests.get(url)

In [7]:
soup = bs(response.text, 'lxml')

In [8]:
links = soup.find_all('a')

In [9]:
print_links = []
for a in links:
    names_and_numbers = {}
    document_id = a['href'].split('/')[-1]
    names_and_numbers['Title'] = a.text.replace(u'\xa0', u' ')
    names_and_numbers['Document_id'] = document_id
    names_and_numbers['print_url'] = print_url+document_id
    print_links.append(names_and_numbers)

In [10]:
print_links

[{'Title': 'MR KC1: Caput standard utredning (MRKC)',
  'Document_id': '14633',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14633'},
 {'Title': 'MR KC2: Caput m/kontrast (MRKCK)',
  'Document_id': '14629',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14629'},
 {'Title': 'MR KC3: Caput Cerebralt infarkt(MRKCINF)',
  'Document_id': '14706',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14706'},
 {'Title': 'MR KC4: Caput demyeliniserende sykdom (MRKCMS)',
  'Document_id': '14708',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14708'},
 {'Title': 'MR KC5: Caput Traume (MRKCTR)',
  'Document_id': '109137',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=109137'},
 {'Title': 'MR KC6: Vaskulitt (MRKCVAS)',
  'Document_id': '127840',
  'print_url': 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=127840'},
 {'Title'

In [11]:
def make_table_data(tables):
    number_of_items = range(0,3)
    rows = tables[5].find_all('tr')
    row_data = []
    for n in number_of_items:
        row_n = []
        for element in rows[1].select('td')[n]:
            row_data = element.get_text(strip=True, separator=',').replace(u'\xa0', u' ')
            row_n.append(row_data)
        clean_list = list(filter(None, row_n))[1:]
        data.append(*clean_list)
    return row_data


In [12]:
link = print_links[0]
print(link['Document_id'])
print_response = requests.get(link['print_url'])
print_soup = bs(print_response.text, 'lxml')
tables = print_soup.find_all('table')
number_of_items = range(0,3)
rows = tables[0].find_all('tr')
row_data = []
for n in number_of_items:
        row_n = []
        for element in rows[1].select('td')[n]:
            row_data = element.get_text(strip=True, separator=',').replace(u'\xa0', u' ')
            row_n.append(row_data)
            print(row_n)
        #clean_list = list(filter(None, row_n))[1:]
        #data.append(*clean_list)


14633
['']


IndexError: list index out of range

In [13]:
def make_table(print_link):
    print_response = requests.get(print_link['print_url'])
    print_soup = bs(print_response.text, 'lxml')
    tables = print_soup.find_all('table')
    rows = tables[2].select('table tr:nth-of-type(9) tr')
    table_row = []
    for row in rows[1:]:
        protocol_row = []
        for value in row.select('td'):
            one_protocol = value.get_text(strip=True, separator=',').replace(u'\xa0', u' ')
            protocol_row.append(one_protocol)
        table_row.append(','.join(protocol_row))
    return table_row

In [24]:
def make_table_test(print_link):
    print_response = requests.get(print_link['print_url'])
    if print_response.ok:
        print_soup = bs(print_response.text, 'lxml')
        tables = print_soup.find_all('table')
        rows = tables[2].select('table tr:nth-of-type(9) tr')
        table_row = []
        if len(rows) > 2:
            for row in rows[1:]:
                protocol_row = []
                for value in row.select('td'):
                    one_protocol = value.get_text(strip=True, separator=',').replace(u'\xa0', u' ')
                    protocol_row.append(one_protocol)
                #table_row.append(','.join(protocol_row))
                while len(protocol_row) < 4:
                    protocol_row.append(' ')
                for item in protocol_row:
                    table_row.append(item)
            return table_row
        else:
            for row in rows[1:]:
                protocol_row = []
                for value in row.select('td'):
                    one_protocol = value.get_text(strip=True, separator=',').replace(u'\xa0', u' ')
                    protocol_row.append(one_protocol)
                if not protocol_row[0] == 'Prekontrast':
                    protocol_row.insert(0, 'Nativ')
                while len(protocol_row) < 4:
                    protocol_row.append(' ')
            return protocol_row

In [15]:
print_response = requests.get(print_links[7]['print_url'])
print_soup = bs(print_response.text, 'lxml')
tables = print_soup.find_all('table td table')
len(tables)

0

In [485]:
make_table_test(print_links[2])

['Nativ',
 'Loc,Sag T1 TSE,Tra T2 4mm,Tra Flair  4mm,Tra SWI,Tra DWI resolve 4mm',
 'Loc,Sag T1 3D mprage,Tra T2 Propeller 4mm,Tra Flair 4mm,Tra SWAN,Tra DWI 4mm',
 'Loc,Sag T1 TSE,Tra T2 4mm,Tra Flair 4mm,Tra SWI/VenBold,Tra DWI 4mm',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [468]:
make_table_test(print_links[1])

['Prekontrast',
 'Loc,Sag T1 3D Space,Tra DWI',
 'Loc,Sag T1 3D Cube,Tra DWI',
 'Loc,Sag T1 3D BrainView,Tra DWI',
 'IV kontrast',
 '',
 '',
 '',
 'Postkontrast',
 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D Space K+',
 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D Cube K+',
 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D BrainView K+']

In [456]:
make_table(print_links[3])

['Prekontrast,Loc,Sag T1 3D Space,Tra DWI,Loc,Sag T1 3D Cube,Tra DWI,Loc,Sag T1 3D BrainView,Tra DWI',
 'IV kontrast,,,',
 'Postkontrast,Tra T2 TSE,Sag 3D Flair FS,Sag T1 3D Space K+,Tra T2 TSE,Sag 3D Flair FS,Sag T1 3D Cube K+,Tra T2 TSE,Sag 3D Flair FS,Sag T1 3D BrainView K+']

In [29]:
table_data = []
for protocol in print_links:
        print(protocol['Title'], protocol['print_url'])
        if 'ANG' in protocol['Title']:
            continue
        elif 'KVAR' in protocol['Title']:
            continue
        elif 'NMR3' in protocol['Title']:
            continue
        elif 'BKCF1' in protocol['Title']:
            continue
        elif 'CTOT1' in protocol['Title']:
            continue
        elif 'CTOT3' in protocol['Title']:
            continue
        elif 'CTOT5' in protocol['Title']:
            continue
        elif 'BORB5' in protocol['Title']:
            continue
        elif 'BORB7' in protocol['Title']:
            continue
        elif protocol['Title'].startswith('MR'):
            table_row = make_table_test(protocol)
            if 'IV' in table_row[0] :
                for n in range(4):
                    table_row.insert(0, '')
            table_row.insert(0, protocol['Title'])
            table_row.insert(1, protocol['print_url'])
            print(table_row)
            table_data.append(table_row)

MR KC1: Caput standard utredning (MRKC) https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14633
['MR KC1: Caput standard utredning (MRKC)', 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14633', 'Nativ', 'Loc,Sag T1 3D mprage,Tra DWI,Sag 3D Flair FS,Tra T2 TSE', 'Loc,Sag 3D T1 mprage,Tra DWI,Sag 3D Flair FS,Tra T2 Propeller', 'Loc,Sag T1 3D  TFE,Tra DWI,Sag 3D Flair FS,Tra T2 TSE']
MR KC2: Caput m/kontrast (MRKCK) https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14629
['MR KC2: Caput m/kontrast (MRKCK)', 'https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14629', 'Prekontrast', 'Loc,Sag T1 3D Space,Tra DWI', 'Loc,Sag T1 3D Cube,Tra DWI', 'Loc,Sag T1 3D BrainView,Tra DWI', 'IV kontrast', '', '', '', 'Postkontrast', 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D Space K+', 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D Cube K+', 'Sag 3D Flair,Tra T2 TSE,Sag T1 3D BrainView K+']
MR KC3: Caput Cerebralt infarkt(MRKCINF) https://ehandboken.ous-hf.no/Printing/

In [28]:
pd.DataFrame(table_data).to_csv('protokoller.csv')

In [415]:
for protocol in print_links[:5]:
    print(protocol['print_url'])

https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14633
https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14629
https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14706
https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=14708
https://ehandboken.ous-hf.no/Printing/PrintDocument?documentId=109137


In [248]:
make_table_data(tables)

['Sag T1 3D mprage,Tra DWI,Sag 3D Flair FS,Tra T2 TSE',
 'Sag 3D T1 mprage,Tra DWI,Sag 3D Flair FS,Tra T2 Propeller',
 'Sag T1 3D  TFE,Tra DWI,Sag 3D Flair FS,Tra T2 TSE']

In [242]:
name = 'hallo'
print(name[:2])

ha


In [239]:
data

['Sag T1 3D  TFE,Tra DWI,Sag 3D Flair FS,Tra T2 TSE']

In [208]:
result = []
for link in print_links:
    one_table = pd.read_html(link['print_url'], header=0)[5]
    result.append(one_table)
all_protocols = pd.concat(result)

KeyboardInterrupt: 

In [ ]:
all_protocols